The problem statement is in the file 'problem statement.txt'.

Explanations and comments on the solution are in the file with the same name.

In [1]:
!pip install \
          transformers==4.35.0 \
          sentencepiece==0.1.99 \
          transformers[sentencepiece] \
          transformers[torch] \
          datasets==2.14.6 \
          peft==0.6.1 --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 52.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 68.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 47.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.0/136.0 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 105.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 108.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 36.5 MB/s eta 0:00:00


In [2]:
from transformers import TrainingArguments, Trainer
from transformers import MT5ForConditionalGeneration, MT5Tokenizer
import torch
import time
import pandas as pd
import numpy as np
import random
from torch.utils.data import Dataset, DataLoader
from tqdm.auto import tqdm
from peft import LoraConfig, get_peft_model, TaskType

In [3]:
SEED = 42

def set_seed(seed=SEED):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(SEED)

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
train_data_tmp = pd.read_csv('train.csv')
val_data_tmp = pd.read_csv('val.csv')
test_data = pd.read_csv('test.csv')

train_data = train_data_tmp[train_data_tmp['label'] == 'entailment']
val_data = val_data_tmp[val_data_tmp['label'] == 'entailment']

#**ОСНОВНАЯ МОДЕЛЬ**

mt5 - Модель которая у меня выступает и в качестве классификатора определяющего entailment и в качестве базы для PEFT / LoRA на основе которого обучается генерация логических следствий.

Ради интереса, я выбрал классификатор обученный не под русский, а под персидский entailment (и дальше по тексту поясняется почему это работает).

https://huggingface.co/persiannlp/mt5-base-parsinlu-snli-entailment

https://github.com/persiannlp/parsinlu/

И этой же моделью я оцениваю качество генерации, что возможно не лучшая идея, но сама по себе оценка генерации логических следствий - нетривильная задача, которая по хорошему требует привлечения человека эксперта, тогда как смысл значений полученных в ходе автоматической оценки таким способом, можно пожалуй понять лишь сравнивая его с оценкой чего то еще.

Также можно было бы использовать ROUGE 1, 2, L как метрику качества, но она бы по качеcтву оценки на такой сложной задаче, вряд ли бы чем то отличалась от выбранного подхода.

In [6]:
model_size="base"
model_name = f"persiannlp/mt5-{model_size}-parsinlu-snli-entailment" # mt5 model for entailment classification

tokenizer = MT5Tokenizer.from_pretrained(model_name)
original_model = MT5ForConditionalGeneration.from_pretrained(model_name, torch_dtype=torch.bfloat16)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
You are using a model of type t5 to instantiate a model of type mt5. This is not supported for all configurations of models and can yield errors.


In [7]:
tokenizerCLS = MT5Tokenizer.from_pretrained(model_name)
modelCLS = MT5ForConditionalGeneration.from_pretrained(model_name, torch_dtype=torch.bfloat16)

You are using a model of type t5 to instantiate a model of type mt5. This is not supported for all configurations of models and can yield errors.


In [8]:
def run_CLS_model(premise, hypothesis, **generator_args):
    input_ids = tokenizerCLS.encode(f"{premise}<sep>{hypothesis}", return_tensors="pt")
    res = modelCLS.generate(input_ids, **generator_args)
    output = tokenizerCLS.batch_decode(res, skip_special_tokens=True)
    print(output)
    return output

# Маленький тест классификатора:
1 ошибка на 10 первых строк в train.csv
и верная классификация на трех гипотезах всех трех типов.

Стоит отдельно отметить, что классификатор на базе мультиязычной mt5, обученный определять entailment на фарси(персидский),
прекрасно справляется с аналогичной задачей на русском,
что находится **в полном согласии** с моей магистерской диссертацией.

**К ВОПРОСУ ПОЧЕМУ ЭТО РАБОТАЕТ:**

То есть, LLMs (и даже такие маленкие как я использовал здесь - 0.5B параметров),умеют формировать на трейне скрытые языковые пространства для разных языков, которые могут быть сопоставлены друг с другом на различных задачах на инференсе (особенно ярко это проявляется на задачах перевода, когда модель не обученная переводу на паре языков (то есть это не языковая пара при обучении),  только на основании 5 качественных few-shot примеров на инференс, начинает переводить на уровне SOTA моделей обученных переводу - статья от Google от февраля 2023 года).

The unreasonable effectiveness of few-shot learning for machine translation
https://arxiv.org/abs/2302.01398

In [9]:
for index, row in train_data_tmp[:10].iterrows():
    print(f'Label = {row["label"]}')
    run_CLS_model(row['premise'], row['hypothesis'])
    print('\n===\n')

Label = entailment


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


['e']

===

Label = not_entailment
['n']

===

Label = not_entailment
['e']

===

Label = entailment
['e']

===

Label = not_entailment
['n']

===

Label = entailment
['e']

===

Label = entailment
['e']

===

Label = not_entailment
['n']

===

Label = entailment
['e']

===

Label = entailment
['e']

===



In [10]:
t = run_CLS_model(
    "Женщину доставили в больницу, за ее жизнь сейчас борются врачи.",
    "Женщину спасают врачи."
)

['e']


In [11]:
t = run_CLS_model(
    "Женщину доставили в больницу, за ее жизнь сейчас борются врачи.",
    "Небо синее"
)

['c']


In [12]:
t = run_CLS_model(
    "Женщину доставили в больницу, за ее жизнь сейчас борются врачи.",
    "Женщина в летах"
)

['n']


--- Конец теста классификатора ---

In [13]:
class TextualEntailmentDataset(Dataset):
    def __init__(self, tokenizer, data_frame, max_length=512):
        self.tokenizer = tokenizer
        self.data = data_frame
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        premise =  row['premise']
        hypothesis = row['hypothesis']

        start_prompt = "Дано высказывание на руском языке: "
        end_prompt = "\nСгенерируй на русском языке логическое следствие из данного высказывания."
        prompt = f'{start_prompt}+[{premise}]+{end_prompt}'

        encoding = self.tokenizer.encode_plus(prompt, add_special_tokens=True,
                                              max_length=self.max_length, padding='max_length',
                                              truncation=True, return_tensors='pt')

        labels = self.tokenizer.encode(hypothesis, add_special_tokens=True,
                                       max_length=self.max_length, padding='max_length',
                                       truncation=True, return_tensors='pt').flatten()
        labels = torch.where(labels != tokenizer.pad_token_id, labels, -100)
        return {
            "input_ids": encoding['input_ids'].squeeze(),
            "attention_mask": encoding['attention_mask'].squeeze(),
            "labels": labels
            }

In [14]:
train_dataset = TextualEntailmentDataset(tokenizer, train_data)
val_dataset = TextualEntailmentDataset(tokenizer, val_data)

In [15]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

In [16]:
lora_config = LoraConfig(
    r=16, # Rank
    #
    # scaling = lora_alpha / r
    # weight += (lora_B @ lora_A) * scaling

    # lora_alpha = sqrt(r) Это мое предположение, по аналогии с делением на корень из внутренней размерности,
    # при перемножении матрицы q и k под softmax в attention.
    # Это приводит std распределения итоговой матрицы после перемножения двух lora матриц к стандартному нормальному

    # SEBASTIAN RASCHKA https://magazine.sebastianraschka.com/p/practical-tips-for-finetuning-llms
    # рекомендует lora_alpha = 2 * r -  в качестве baseline.
    # Но по факту рекомендация просто пробовать, может оказаться сильно по разному в зависимости от задачи,
    # в статье есть табличка с экспериментами в ответе на вопрос № 7.
    #
    lora_alpha=4,
    target_modules=["q", "v", 'k', 'o', 'wi_0', 'wi_1', 'wo'],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM
)

In [17]:
peft_model = get_peft_model(original_model,
                            lora_config)
print(print_number_of_trainable_model_parameters(peft_model))

trainable model parameters: 6782976
all model parameters: 589184256
percentage of trainable model parameters: 1.15%


In [18]:
peft_model

PeftModelForSeq2SeqLM(
  (base_model): LoraModel(
    (model): MT5ForConditionalGeneration(
      (shared): Embedding(250112, 768)
      (encoder): MT5Stack(
        (embed_tokens): Embedding(250112, 768)
        (block): ModuleList(
          (0): MT5Block(
            (layer): ModuleList(
              (0): MT5LayerSelfAttention(
                (SelfAttention): MT5Attention(
                  (q): Linear(
                    in_features=768, out_features=768, bias=False
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.05, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=768, out_features=16, bias=False)
                    )
                    (lora_B): ModuleDict(
                      (default): Linear(in_features=16, out_features=768, bias=False)
                    )
                    (lora_embedding_A): ParameterDict()
                    (lora_embedd

In [19]:
from transformers import TrainerCallback

In [20]:
output_dir = f'./peft'

peft_training_args = TrainingArguments(
    output_dir=output_dir,
    auto_find_batch_size=True,
    learning_rate=1e-3,
    num_train_epochs=10,
    logging_dir='./logs',
    logging_steps=100,
    evaluation_strategy="steps",
    eval_steps=100,
    max_steps= 2000

)

peft_trainer = Trainer(
    model=peft_model,
    args=peft_training_args,
    train_dataset=train_dataset,
    eval_dataset = val_dataset
)

In [21]:
peft_trainer.train()

Step,Training Loss,Validation Loss
100,5.989000,2.002757
200,2.878700,1.853145
300,2.693900,1.838899
400,2.515100,1.778697
500,2.408200,1.694955
600,2.477000,1.682190
700,2.320400,1.698325
800,2.268400,1.690104
900,2.215500,1.731107
1000,2.252300,1.649229


TrainOutput(global_step=2000, training_loss=2.425599609375, metrics={'train_runtime': 4446.252, 'train_samples_per_second': 1.799, 'train_steps_per_second': 0.45, 'total_flos': 9752968054702080.0, 'train_loss': 2.425599609375, 'epoch': 5.88})

Сохранение только LoRA весов:

In [22]:
def save_lora_weights(model, save_path):
    lora_weights = {}
    for name, module in model.named_modules():
        if hasattr(module, 'lora_A') and hasattr(module, 'lora_B'):
            lora_weights[f'{name}_lora_A'] = getattr(module, 'lora_A').state_dict()
            lora_weights[f'{name}_lora_B'] = getattr(module, 'lora_B').state_dict()
    torch.save(lora_weights, save_path)

In [23]:
save_lora_weights(peft_model, 'lora_weights.pth')

Восстановление LoRA весов:

In [24]:
def load_lora_weights(model, load_path):
    lora_weights = torch.load(load_path)
    for name, module in model.named_modules():
        if hasattr(module, 'lora_A') and hasattr(module, 'lora_B'):
            module.lora_A.load_state_dict(lora_weights[f'{name}_lora_A'])
            module.lora_B.load_state_dict(lora_weights[f'{name}_lora_B'])

In [25]:
# load_lora_weights(peft_model, 'lora_weights.pth')

In [26]:
# For all weights peft_model
# torch.save(peft_model.state_dict(), 'peft_model_weights.pth')
#
# new_peft_model = ... # with the same configuration of model
# new_peft_model.load_state_dict(torch.load('peft_model_weights.pth'))

In [27]:
def tokenize_premises(tokenizer, premises, max_length=512):
    return tokenizer(premises, add_special_tokens=True, return_tensors="pt",
                     padding="max_length", truncation=True, max_length=max_length)

def generate_predictions_for_test_set(model, tokenizer, test_data, device, num_predictions=10, max_length=512, temperature = 1.0):
    model.eval()
    premises = test_data['premise'].tolist()
    result = test_data['premise'].tolist()
    start_prompt = "Дано высказывание на руском языке: "
    end_prompt = "\nСгенерируй на русском языке логическое следствие из данного высказывания."
    list_of_prompts = []
    for premise in result:
        list_of_prompts.append(f'{start_prompt} + [{premise}] + {end_prompt}')
    encoded_input = tokenize_premises(tokenizer, list_of_prompts, max_length=max_length)
    predictions = []
    total_iter_num = 0
    total_bad_num = 0
    for index in tqdm(range(len(premises)), desc="Generating predictions"):
        print(f'Premise # {index}')
        input_ids = encoded_input['input_ids'][index].unsqueeze(0).to(device)
        attention_mask = encoded_input['attention_mask'][index].unsqueeze(0).to(device)
        preds10 = []
        count = num_predictions
        while count != 0:
            pred = model.generate(input_ids=input_ids, attention_mask=attention_mask,
                                  max_length=max_length, do_sample=True,
                                  num_return_sequences=1) #, temperature = temperature, num_predictions)
            pred = tokenizer.decode(pred[0], skip_special_tokens=True)
            result_CLS = run_CLS_model(premises[index], pred)

            if pred in preds10:
                print("It was already on the list")
            else:
                total_iter_num += 1
                if result_CLS[0] == 'e':
                    preds10.append(pred)
                    count -= 1
                else:
                    total_bad_num += 1

        predictions.append(preds10)
    total_good_num = total_iter_num-total_bad_num
    print('\n=====\n')
    print(f'Generation quality: {(total_iter_num-total_bad_num)/total_iter_num}')
    print(f'Classified as entailment(according to CLS model): {total_good_num} from all generation: {total_iter_num}')
    print('\n=====\n')
    return predictions

In [28]:
test_predictions1_0 = generate_predictions_for_test_set(peft_model, tokenizer, test_data, device, temperature = 1.0)
test_data['predictions'] = test_predictions1_0
test_data.to_csv('test.csv', index=False)
for index, row in test_data.iterrows():
    premise = row['premise']
    print(f'Premise: {premise}')
    print('Predictions:')
    for pred in row['predictions']:
        print(f'  - {pred}')
    print("\n")

Generating predictions:   0%|          | 0/5 [00:00<?, ?it/s]

Premise # 0


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


['e']
['n']
['e']
['e']
['e']
['e']
['e']
['n']
['e']
['e']
['e']
['n']
['e']
Premise # 1
['e']
['e']
['n']
['e']
['e']
['e']
['n']
['e']
['e']
['e']
['e']
['e']
Premise # 2
['e']
['e']
['e']
['e']
['c']
['e']
['e']
['e']
['n']
['e']
['e']
['e']
Premise # 3
['e']
['n']
['c']
['c']
['c']
['n']
['e']
['c']
['e']
['n']
['n']
['e']
['e']
['e']
['e']
['n']
['c']
['c']
['c']
['e']
['n']
['e']
['n']
['e']
Premise # 4
['e']
['e']
['e']
['n']
['e']
['e']
['e']
['e']
['n']
['e']
['e']
['e']

=====

Generation quality: 0.684931506849315
Classified as entailment(according to CLS model): 50 from all generation: 73

=====

Premise: Готовившие госпереворот в Германии планировали убийство канцлера ФРГ Олафа Шольца, сообщает The New York Times со ссылкой на источники, близкие к расследованию. В ходе обысков сотрудники полиции обнаружили более ста соглашений о неразглашении, в которых заговорщики поклялись сохранять секретность планов группы. В них входили нападение и арест членов парламента, а также уб

Качество генерации по мнению классифкатора: 0.69

Генерация дублей при подсчете не учитывалась(их и не было при данных параметрах), 50 "entailment" среди 73 оригинальных генераций.

Эти цифры есть в выдаче - перед сгенерированными списками гипотез.



Да, надо полагать, что модель очень специфична в оценках собственной генерации - что то близкое к принципу используему в ROUGE, скорей даже просто сопадение слов из исходного посыла.

В реальной задаче стоило бы, для чистоты эксперимента взять другой классификатор - лучше большую LLM, для оценки качества  генерации, тогда оно бы было очень низким.


